# Preparação do ambiente

## Bibliotecas

In [2]:
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import os.path as osp
import pandas as pd
import re
import unidecode

from datetime import datetime
from joblib import dump

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline

from sklearn.dummy import DummyClassifier

from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import MultinomialNB

from sklearn.neighbors import KNeighborsClassifier

from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier

from sklearn.svm import LinearSVC
from sklearn.svm import NuSVC

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

from string import punctuation

In [3]:
import warnings
warnings.filterwarnings("ignore") 
if os.name=='posix':
    os.environ['PYTHONWARNINGS']='ignore'

## Constantes e funções auxiliares

In [4]:
FILE_MARKETPLACES = '../datasets/samples_labeling/issues_fiscaliza/20240423/results/labels.parquet'
FILE_SUPERVISAO_MERCADO = '../datasets/inspecao_ecommerce/supervisao_mercado.xlsx'
FILE_HYPER_PARAMETERS = '../datasets/experimento_gs/gs_hyper_paramenters_classifiers.json'

N_JOBS = os.cpu_count()//2

In [5]:
def load_file_mercado(file_mercado=FILE_SUPERVISAO_MERCADO):
    dict_df_mercado = pd.read_excel(
    file_mercado,
    sheet_name=None,
    usecols=[2,8,10],
    names=['texto_busca', 'titulo', 'passivel_homologacao'],
    true_values=['Sim', 'sim'],
    false_values=['Não','não'],
    na_values=['-'])

    df_list = []
    for key in dict_df_mercado.keys():
        df = dict_df_mercado[key]
        df['marketplace'] = key
        df_list.append(df)
        
    df_mercado = pd.concat(df_list)
    df_mercado = df_mercado.dropna()
    df_mercado['passivel_homologacao'] = df_mercado['passivel_homologacao'].astype(int)
    
    map_marketplaces = {
        'Amazon': 'Amazon', 
        'Americanas': 'Lojas Americanas',
        'CasasBahia': 'Casas Bahia',
        'Magalu': 'Magazine Luiza', 
        'MercadoLivre': 'Mercado Livre'
    }
    
    df_mercado['marketplace'] = df_mercado['marketplace'].map(map_marketplaces)

    return df_mercado

In [6]:
def save_best_params():
    
    _best_params = {
        'name': gs.best_estimator_['clf'].__class__.__name__,
        'best_params': json.dumps(gs.best_params_),
        'mean_fit_time': gs.cv_results_['mean_fit_time'].mean(),
        'timestamp': datetime.now().timestamp(),
        'train_auc': gs.best_score_,
        'train_accuracy': gs.score(X_test,y_test),
    }

    # test accuracy and roc_auc_score
    test_accuracy = gs.best_estimator_.score(X_test,y_test)
    valid_accuracy = gs.best_estimator_.score(X_valid,y_valid)
    
    if hasattr(gs.best_estimator_['clf'],'predict_proba'):
        y_test_score = gs.best_estimator_.predict_proba(X_test)[:,1]
        y_valid_score = gs.best_estimator_.predict_proba(X_valid)[:,1]
    else:
        y_test_score = gs.best_estimator_.decision_function(X_test)
        y_valid_score = gs.best_estimator_.decision_function(X_valid)
        
    test_auc = roc_auc_score(y_test,y_test_score)
    valid_auc = roc_auc_score(y_valid,y_valid_score)

    _best_params['test_auc'] = test_auc
    _best_params['test_accuracy'] = test_accuracy

    _best_params['valid_auc'] = valid_auc
    _best_params['valid_accuracy'] = valid_accuracy

    # confusion matrix
    y_pred = gs.predict(X_valid)
    cm = confusion_matrix(y_valid,y_pred)
    tn = int(cm[0][0])
    fn = int(cm[1][0])
    tp = int(cm[1][1])
    fp = int(cm[0][1])
    
    confusion = {
        'tn': tn, 
        'fn': fn, 
        'tp': tp, 
        'fp': fp
    }

    _best_params['confusion'] = json.dumps(confusion)
    
    if osp.exists(FILE_HYPER_PARAMETERS):
        with open(FILE_HYPER_PARAMETERS, 'r') as f:
            clf_best_params = json.load(f) 
    else:
        clf_best_params = []
    
    clf_best_params.append(_best_params)
    
    df = pd.DataFrame(clf_best_params)
    df = df.sort_values(by=['test_auc','test_accuracy', 'timestamp'], ascending=False).reset_index(drop=True)
    df = df.drop_duplicates(subset='name',keep='first')

    # save best params
    clf_best_params = df.to_dict('records')
    with open(FILE_HYPER_PARAMETERS, 'w+') as f:
        json.dump(clf_best_params,f,indent=2)

    # save model
    pipe = gs.best_estimator_
    clf_name = pipe['clf'].__class__.__name__
    clf_joblib_file = f'../datasets/experimento_gs/models/{clf_name}.joblib'
    with open(clf_joblib_file,'wb') as f:
        dump(pipe, f)
    
    return df

In [7]:
def load_best_params():

    with open(FILE_HYPER_PARAMETERS) as f:
        clf_best_params = json.load(f)
        
    return pd.DataFrame(clf_best_params)

# Carga e prepação dos dados

In [8]:
df_marketplaces = pd.read_parquet(FILE_MARKETPLACES)
df_mercado = load_file_mercado()

# reduzir a base de dados para construir o notebook
# df_marketplaces = df_marketplaces.sample(10)

# conjunto de dados
docs_marketplaces = df_marketplaces[df_marketplaces['passivel_homologacao']<2]['titulo'].values
targets_marketplaces = df_marketplaces[df_marketplaces['passivel_homologacao']<2]['passivel_homologacao'].values

# Split data to keep experiment results comparable
X_train, X_test, y_train, y_test = train_test_split(
    docs_marketplaces, targets_marketplaces,
    test_size=0.25, 
    random_state=724
)

X_valid, y_valid = df_mercado['titulo'].values, df_mercado['passivel_homologacao'].values

# Análise


https://levity.ai/blog/text-classifiers-in-machine-learning-a-practical-guide

## Ajustando hiperparâmetros dos algoritmos

In [9]:
start_time = datetime.now()

### DummyClassifier

In [10]:
%%time
clf = DummyClassifier()

vectorizer = TfidfVectorizer()
pipe = Pipeline(steps = [('vectorizer',vectorizer),('clf',clf)])

parameter_grid = {
    "clf__strategy": ('most_frequent', 'prior', 'stratified', 'uniform', 'constant'),
}
scoring = {"AUC": "roc_auc"}

gs = GridSearchCV(pipe,parameter_grid,scoring=scoring,refit='AUC', n_jobs=N_JOBS)
gs.fit(X_train, y_train)
save_best_params()

CPU times: total: 1.36 s
Wall time: 15.6 s


,name,best_params,mean_fit_time,timestamp,train_auc,train_accuracy,test_auc,test_accuracy,valid_auc,valid_accuracy,confusion
0,BernoulliNB,"{""clf__alpha"": 1.0}",0.268706,1.743708e+09,0.956423,0.950095,0.950095,0.891701,0.947094,0.880615,"{""tn"": 69, ""fn"": 7, ""tp"": 676, ""fp"": 94}"
1,DummyClassifier,"{""clf__strategy"": ""most_frequent""}",0.445810,1.743708e+09,0.500000,0.500000,0.500000,0.577519,0.500000,0.807329,"{""tn"": 0, ""fn"": 0, ""tp"": 683, ""fp"": 163}"


### NaiveBayes

#### BernoulliNB

In [11]:
%%time
clf = BernoulliNB()

vectorizer = TfidfVectorizer()
pipe = Pipeline(steps = [('vectorizer',vectorizer),('clf',clf)])

parameter_grid = {
    "clf__alpha": np.logspace(-6, 6, 13),
}
scoring = {"AUC": "roc_auc"}

gs = GridSearchCV(pipe,parameter_grid,scoring=scoring,refit='AUC', n_jobs=N_JOBS)
gs.fit(X_train, y_train)

save_best_params()

CPU times: total: 1.78 s
Wall time: 7.82 s


,name,best_params,mean_fit_time,timestamp,train_auc,train_accuracy,test_auc,test_accuracy,valid_auc,valid_accuracy,confusion
0,BernoulliNB,"{""clf__alpha"": 1.0}",0.457339,1.743708e+09,0.956423,0.950095,0.950095,0.891701,0.947094,0.880615,"{""tn"": 69, ""fn"": 7, ""tp"": 676, ""fp"": 94}"
2,DummyClassifier,"{""clf__strategy"": ""most_frequent""}",0.445810,1.743708e+09,0.500000,0.500000,0.500000,0.577519,0.500000,0.807329,"{""tn"": 0, ""fn"": 0, ""tp"": 683, ""fp"": 163}"


#### ComplementNB

In [12]:
%%time
clf = ComplementNB()

vectorizer = TfidfVectorizer()
pipe = Pipeline(steps = [('vectorizer',vectorizer),('clf',clf)])

parameter_grid = {
    "clf__alpha": np.logspace(-6, 6, 13),
}
scoring = {"AUC": "roc_auc"}

gs = GridSearchCV(pipe,parameter_grid,scoring=scoring,refit='AUC', n_jobs=N_JOBS)
gs.fit(X_train, y_train)
save_best_params()

CPU times: total: 1.73 s
Wall time: 8.76 s


,name,best_params,mean_fit_time,timestamp,train_auc,train_accuracy,test_auc,test_accuracy,valid_auc,valid_accuracy,confusion
0,ComplementNB,"{""clf__alpha"": 1.0}",0.543166,1.743708e+09,0.959284,0.955167,0.955167,0.893981,0.953058,0.880615,"{""tn"": 72, ""fn"": 10, ""tp"": 673, ""fp"": 91}"
1,BernoulliNB,"{""clf__alpha"": 1.0}",0.457339,1.743708e+09,0.956423,0.950095,0.950095,0.891701,0.947094,0.880615,"{""tn"": 69, ""fn"": 7, ""tp"": 676, ""fp"": 94}"
2,DummyClassifier,"{""clf__strategy"": ""most_frequent""}",0.445810,1.743708e+09,0.500000,0.500000,0.500000,0.577519,0.500000,0.807329,"{""tn"": 0, ""fn"": 0, ""tp"": 683, ""fp"": 163}"


#### MultinomialNB

In [13]:
%%time
clf = MultinomialNB()

vectorizer = TfidfVectorizer()
pipe = Pipeline(steps = [('vectorizer',vectorizer),('clf',clf)])

parameter_grid = {
    "clf__alpha": np.logspace(-6, 6, 13),
}
scoring = {"AUC": "roc_auc"}

gs = GridSearchCV(pipe, parameter_grid, scoring=scoring, refit='AUC', n_jobs=N_JOBS)
gs.fit(X_train, y_train)
save_best_params()

CPU times: total: 1.77 s
Wall time: 8.14 s


,name,best_params,mean_fit_time,timestamp,train_auc,train_accuracy,test_auc,test_accuracy,valid_auc,valid_accuracy,confusion
0,ComplementNB,"{""clf__alpha"": 1.0}",0.543166,1.743708e+09,0.959284,0.955167,0.955167,0.893981,0.953058,0.880615,"{""tn"": 72, ""fn"": 10, ""tp"": 673, ""fp"": 91}"
1,MultinomialNB,"{""clf__alpha"": 1.0}",0.471005,1.743708e+09,0.959284,0.955167,0.955167,0.886001,0.953058,0.877069,"{""tn"": 61, ""fn"": 2, ""tp"": 681, ""fp"": 102}"
2,BernoulliNB,"{""clf__alpha"": 1.0}",0.457339,1.743708e+09,0.956423,0.950095,0.950095,0.891701,0.947094,0.880615,"{""tn"": 69, ""fn"": 7, ""tp"": 676, ""fp"": 94}"
3,DummyClassifier,"{""clf__strategy"": ""most_frequent""}",0.445810,1.743708e+09,0.500000,0.500000,0.500000,0.577519,0.500000,0.807329,"{""tn"": 0, ""fn"": 0, ""tp"": 683, ""fp"": 163}"


### KNeighborsClassifier

In [14]:
%%time
clf = KNeighborsClassifier()

vectorizer = TfidfVectorizer()
pipe = Pipeline(steps = [('vectorizer',vectorizer),('clf',clf)])

parameter_grid = {
    "clf__n_neighbors": (3, 5, 10),
    "clf__p": (1, 2)
}
scoring = {"AUC": "roc_auc"}

gs = GridSearchCV(pipe,parameter_grid,scoring=scoring,refit='AUC', n_jobs=N_JOBS)
gs.fit(X_train, y_train)
save_best_params()

CPU times: total: 54 s
Wall time: 44.2 s


,name,best_params,mean_fit_time,timestamp,train_auc,train_accuracy,test_auc,test_accuracy,valid_auc,valid_accuracy,confusion
0,ComplementNB,"{""clf__alpha"": 1.0}",0.543166,1.743708e+09,0.959284,0.955167,0.955167,0.893981,0.953058,0.880615,"{""tn"": 72, ""fn"": 10, ""tp"": 673, ""fp"": 91}"
1,MultinomialNB,"{""clf__alpha"": 1.0}",0.471005,1.743708e+09,0.959284,0.955167,0.955167,0.886001,0.953058,0.877069,"{""tn"": 61, ""fn"": 2, ""tp"": 681, ""fp"": 102}"
2,KNeighborsClassifier,"{""clf__n_neighbors"": 10, ""clf__p"": 2}",0.548373,1.743708e+09,0.955173,0.951467,0.951467,0.895121,0.898917,0.880615,"{""tn"": 79, ""fn"": 17, ""tp"": 666, ""fp"": 84}"
3,BernoulliNB,"{""clf__alpha"": 1.0}",0.457339,1.743708e+09,0.956423,0.950095,0.950095,0.891701,0.947094,0.880615,"{""tn"": 69, ""fn"": 7, ""tp"": 676, ""fp"": 94}"
4,DummyClassifier,"{""clf__strategy"": ""most_frequent""}",0.445810,1.743708e+09,0.500000,0.500000,0.500000,0.577519,0.500000,0.807329,"{""tn"": 0, ""fn"": 0, ""tp"": 683, ""fp"": 163}"


### Linear models

#### SGDClassifier

In [15]:
%%time
clf = SGDClassifier()

vectorizer = TfidfVectorizer()
pipe = Pipeline(steps = [('vectorizer',vectorizer),('clf',clf)])

parameter_grid = {
    "clf__alpha": np.logspace(-6, 6, 13),
    "clf__loss": ('hinge', 'log_loss', 'modified_huber', 'squared_hinge', 'perceptron'),
    "clf__penalty": ('l1', 'l2', 'elasticnet'),
}
scoring = {"AUC": "roc_auc"}

gs = GridSearchCV(pipe,parameter_grid,scoring=scoring,refit='AUC', n_jobs=N_JOBS)
gs.fit(X_train, y_train)
save_best_params()

CPU times: total: 12.2 s
Wall time: 1min 49s


,name,best_params,mean_fit_time,timestamp,train_auc,train_accuracy,test_auc,test_accuracy,valid_auc,valid_accuracy,confusion
0,SGDClassifier,"{""clf__alpha"": 1e-05, ""clf__loss"": ""log_loss"",...",0.549673,1.743709e+09,0.967128,0.967872,0.967872,0.911993,0.962193,0.905437,"{""tn"": 95, ""fn"": 12, ""tp"": 671, ""fp"": 68}"
1,ComplementNB,"{""clf__alpha"": 1.0}",0.543166,1.743708e+09,0.959284,0.955167,0.955167,0.893981,0.953058,0.880615,"{""tn"": 72, ""fn"": 10, ""tp"": 673, ""fp"": 91}"
2,MultinomialNB,"{""clf__alpha"": 1.0}",0.471005,1.743708e+09,0.959284,0.955167,0.955167,0.886001,0.953058,0.877069,"{""tn"": 61, ""fn"": 2, ""tp"": 681, ""fp"": 102}"
3,KNeighborsClassifier,"{""clf__n_neighbors"": 10, ""clf__p"": 2}",0.548373,1.743708e+09,0.955173,0.951467,0.951467,0.895121,0.898917,0.880615,"{""tn"": 79, ""fn"": 17, ""tp"": 666, ""fp"": 84}"
4,BernoulliNB,"{""clf__alpha"": 1.0}",0.457339,1.743708e+09,0.956423,0.950095,0.950095,0.891701,0.947094,0.880615,"{""tn"": 69, ""fn"": 7, ""tp"": 676, ""fp"": 94}"
5,DummyClassifier,"{""clf__strategy"": ""most_frequent""}",0.445810,1.743708e+09,0.500000,0.500000,0.500000,0.577519,0.500000,0.807329,"{""tn"": 0, ""fn"": 0, ""tp"": 683, ""fp"": 163}"


#### LogisticRegression

In [16]:
%%time
clf = LogisticRegression(solver='liblinear')

vectorizer = TfidfVectorizer()
pipe = Pipeline(steps = [('vectorizer',vectorizer),('clf',clf)])

parameter_grid = {
    "clf__C": np.logspace(-5, 5, 11),
    "clf__penalty": ('l1', 'l2'),
}
scoring = {"AUC": "roc_auc"}

gs = GridSearchCV(pipe,parameter_grid,scoring=scoring,refit='AUC', n_jobs=N_JOBS)
gs.fit(X_train, y_train)
save_best_params()

CPU times: total: 3.78 s
Wall time: 18.5 s


,name,best_params,mean_fit_time,timestamp,train_auc,train_accuracy,test_auc,test_accuracy,valid_auc,valid_accuracy,confusion
0,SGDClassifier,"{""clf__alpha"": 1e-05, ""clf__loss"": ""log_loss"",...",0.549673,1.743709e+09,0.967128,0.967872,0.967872,0.911993,0.962193,0.905437,"{""tn"": 95, ""fn"": 12, ""tp"": 671, ""fp"": 68}"
1,LogisticRegression,"{""clf__C"": 10.0, ""clf__penalty"": ""l2""}",0.749055,1.743709e+09,0.967533,0.967745,0.967745,0.911537,0.962337,0.903073,"{""tn"": 93, ""fn"": 12, ""tp"": 671, ""fp"": 70}"
2,ComplementNB,"{""clf__alpha"": 1.0}",0.543166,1.743708e+09,0.959284,0.955167,0.955167,0.893981,0.953058,0.880615,"{""tn"": 72, ""fn"": 10, ""tp"": 673, ""fp"": 91}"
3,MultinomialNB,"{""clf__alpha"": 1.0}",0.471005,1.743708e+09,0.959284,0.955167,0.955167,0.886001,0.953058,0.877069,"{""tn"": 61, ""fn"": 2, ""tp"": 681, ""fp"": 102}"
4,KNeighborsClassifier,"{""clf__n_neighbors"": 10, ""clf__p"": 2}",0.548373,1.743708e+09,0.955173,0.951467,0.951467,0.895121,0.898917,0.880615,"{""tn"": 79, ""fn"": 17, ""tp"": 666, ""fp"": 84}"
5,BernoulliNB,"{""clf__alpha"": 1.0}",0.457339,1.743708e+09,0.956423,0.950095,0.950095,0.891701,0.947094,0.880615,"{""tn"": 69, ""fn"": 7, ""tp"": 676, ""fp"": 94}"
6,DummyClassifier,"{""clf__strategy"": ""most_frequent""}",0.445810,1.743708e+09,0.500000,0.500000,0.500000,0.577519,0.500000,0.807329,"{""tn"": 0, ""fn"": 0, ""tp"": 683, ""fp"": 163}"


#### RidgeClassifier

In [17]:
%%time
clf = RidgeClassifier(fit_intercept=False)

vectorizer = TfidfVectorizer()
pipe = Pipeline(steps = [('vectorizer',vectorizer),('clf',clf)])

parameter_grid = {
    "clf__solver": ('cholesky', 'sparse_cg', 'sag', 'saga'),
}
scoring = {"AUC": "roc_auc"}

gs = GridSearchCV(pipe,parameter_grid,scoring=scoring,refit='AUC', n_jobs=N_JOBS)
gs.fit(X_train, y_train)
save_best_params()

CPU times: total: 42.2 s
Wall time: 1min 2s


,name,best_params,mean_fit_time,timestamp,train_auc,train_accuracy,test_auc,test_accuracy,valid_auc,valid_accuracy,confusion
0,SGDClassifier,"{""clf__alpha"": 1e-05, ""clf__loss"": ""log_loss"",...",0.549673,1.743709e+09,0.967128,0.967872,0.967872,0.911993,0.962193,0.905437,"{""tn"": 95, ""fn"": 12, ""tp"": 671, ""fp"": 68}"
1,LogisticRegression,"{""clf__C"": 10.0, ""clf__penalty"": ""l2""}",0.749055,1.743709e+09,0.967533,0.967745,0.967745,0.911537,0.962337,0.903073,"{""tn"": 93, ""fn"": 12, ""tp"": 671, ""fp"": 70}"
2,RidgeClassifier,"{""clf__solver"": ""cholesky""}",11.230238,1.743709e+09,0.965955,0.965536,0.965536,0.911309,0.951208,0.893617,"{""tn"": 88, ""fn"": 15, ""tp"": 668, ""fp"": 75}"
3,ComplementNB,"{""clf__alpha"": 1.0}",0.543166,1.743708e+09,0.959284,0.955167,0.955167,0.893981,0.953058,0.880615,"{""tn"": 72, ""fn"": 10, ""tp"": 673, ""fp"": 91}"
4,MultinomialNB,"{""clf__alpha"": 1.0}",0.471005,1.743708e+09,0.959284,0.955167,0.955167,0.886001,0.953058,0.877069,"{""tn"": 61, ""fn"": 2, ""tp"": 681, ""fp"": 102}"
5,KNeighborsClassifier,"{""clf__n_neighbors"": 10, ""clf__p"": 2}",0.548373,1.743708e+09,0.955173,0.951467,0.951467,0.895121,0.898917,0.880615,"{""tn"": 79, ""fn"": 17, ""tp"": 666, ""fp"": 84}"
6,BernoulliNB,"{""clf__alpha"": 1.0}",0.457339,1.743708e+09,0.956423,0.950095,0.950095,0.891701,0.947094,0.880615,"{""tn"": 69, ""fn"": 7, ""tp"": 676, ""fp"": 94}"
7,DummyClassifier,"{""clf__strategy"": ""most_frequent""}",0.445810,1.743708e+09,0.500000,0.500000,0.500000,0.577519,0.500000,0.807329,"{""tn"": 0, ""fn"": 0, ""tp"": 683, ""fp"": 163}"


### SVM

#### LinearSVC

In [18]:
%%time
clf = LinearSVC()

vectorizer = TfidfVectorizer()
pipe = Pipeline(steps = [('vectorizer',vectorizer),('clf',clf)])

parameter_grid = {
    "clf__loss": ('hinge', 'squared_hinge'),
    "clf__C": np.logspace(-5,0,6),
    "clf__tol": np.logspace(-5,0,6),
}
scoring = {"AUC": "roc_auc"}

gs = GridSearchCV(pipe,parameter_grid,scoring=scoring,refit='AUC', n_jobs=N_JOBS)
gs.fit(X_train, y_train)
save_best_params()

CPU times: total: 4.28 s
Wall time: 36.8 s


,name,best_params,mean_fit_time,timestamp,train_auc,train_accuracy,test_auc,test_accuracy,valid_auc,valid_accuracy,confusion
0,SGDClassifier,"{""clf__alpha"": 1e-05, ""clf__loss"": ""log_loss"",...",0.549673,1.743709e+09,0.967128,0.967872,0.967872,0.911993,0.962193,0.905437,"{""tn"": 95, ""fn"": 12, ""tp"": 671, ""fp"": 68}"
1,LogisticRegression,"{""clf__C"": 10.0, ""clf__penalty"": ""l2""}",0.749055,1.743709e+09,0.967533,0.967745,0.967745,0.911537,0.962337,0.903073,"{""tn"": 93, ""fn"": 12, ""tp"": 671, ""fp"": 70}"
2,LinearSVC,"{""clf__C"": 0.1, ""clf__loss"": ""squared_hinge"", ...",0.464070,1.743709e+09,0.966799,0.965810,0.965810,0.908801,0.949636,0.881797,"{""tn"": 74, ""fn"": 11, ""tp"": 672, ""fp"": 89}"
3,RidgeClassifier,"{""clf__solver"": ""cholesky""}",11.230238,1.743709e+09,0.965955,0.965536,0.965536,0.911309,0.951208,0.893617,"{""tn"": 88, ""fn"": 15, ""tp"": 668, ""fp"": 75}"
4,ComplementNB,"{""clf__alpha"": 1.0}",0.543166,1.743708e+09,0.959284,0.955167,0.955167,0.893981,0.953058,0.880615,"{""tn"": 72, ""fn"": 10, ""tp"": 673, ""fp"": 91}"
5,MultinomialNB,"{""clf__alpha"": 1.0}",0.471005,1.743708e+09,0.959284,0.955167,0.955167,0.886001,0.953058,0.877069,"{""tn"": 61, ""fn"": 2, ""tp"": 681, ""fp"": 102}"
6,KNeighborsClassifier,"{""clf__n_neighbors"": 10, ""clf__p"": 2}",0.548373,1.743708e+09,0.955173,0.951467,0.951467,0.895121,0.898917,0.880615,"{""tn"": 79, ""fn"": 17, ""tp"": 666, ""fp"": 84}"
7,BernoulliNB,"{""clf__alpha"": 1.0}",0.457339,1.743708e+09,0.956423,0.950095,0.950095,0.891701,0.947094,0.880615,"{""tn"": 69, ""fn"": 7, ""tp"": 676, ""fp"": 94}"
8,DummyClassifier,"{""clf__strategy"": ""most_frequent""}",0.445810,1.743708e+09,0.500000,0.500000,0.500000,0.577519,0.500000,0.807329,"{""tn"": 0, ""fn"": 0, ""tp"": 683, ""fp"": 163}"


#### NuSVC

In [19]:
%%time
clf = NuSVC()

vectorizer = TfidfVectorizer()
pipe = Pipeline(steps = [('vectorizer',vectorizer),('clf',clf)])

parameter_grid = {
    "clf__nu": (0.1, 0.25, 0.5, 0.75, 1),
    "clf__kernel": ('linear', 'poly', 'rbf', 'sigmoid', 'precomputed'),
    "clf__tol": np.logspace(-5,0,6),
}
scoring = {"AUC": "roc_auc"}

gs = GridSearchCV(pipe,parameter_grid,scoring=scoring,refit='AUC', n_jobs=N_JOBS)
gs.fit(X_train, y_train)
save_best_params()

CPU times: total: 1min 15s
Wall time: 46min 51s


,name,best_params,mean_fit_time,timestamp,train_auc,train_accuracy,test_auc,test_accuracy,valid_auc,valid_accuracy,confusion
0,NuSVC,"{""clf__kernel"": ""rbf"", ""clf__nu"": 0.25, ""clf__...",19.793394,1.743712e+09,0.970584,0.968602,0.968602,0.919745,0.962193,0.904255,"{""tn"": 97, ""fn"": 15, ""tp"": 668, ""fp"": 66}"
1,SGDClassifier,"{""clf__alpha"": 1e-05, ""clf__loss"": ""log_loss"",...",0.549673,1.743709e+09,0.967128,0.967872,0.967872,0.911993,0.962193,0.905437,"{""tn"": 95, ""fn"": 12, ""tp"": 671, ""fp"": 68}"
2,LogisticRegression,"{""clf__C"": 10.0, ""clf__penalty"": ""l2""}",0.749055,1.743709e+09,0.967533,0.967745,0.967745,0.911537,0.962337,0.903073,"{""tn"": 93, ""fn"": 12, ""tp"": 671, ""fp"": 70}"
3,LinearSVC,"{""clf__C"": 0.1, ""clf__loss"": ""squared_hinge"", ...",0.464070,1.743709e+09,0.966799,0.965810,0.965810,0.908801,0.949636,0.881797,"{""tn"": 74, ""fn"": 11, ""tp"": 672, ""fp"": 89}"
4,RidgeClassifier,"{""clf__solver"": ""cholesky""}",11.230238,1.743709e+09,0.965955,0.965536,0.965536,0.911309,0.951208,0.893617,"{""tn"": 88, ""fn"": 15, ""tp"": 668, ""fp"": 75}"
5,ComplementNB,"{""clf__alpha"": 1.0}",0.543166,1.743708e+09,0.959284,0.955167,0.955167,0.893981,0.953058,0.880615,"{""tn"": 72, ""fn"": 10, ""tp"": 673, ""fp"": 91}"
6,MultinomialNB,"{""clf__alpha"": 1.0}",0.471005,1.743708e+09,0.959284,0.955167,0.955167,0.886001,0.953058,0.877069,"{""tn"": 61, ""fn"": 2, ""tp"": 681, ""fp"": 102}"
7,KNeighborsClassifier,"{""clf__n_neighbors"": 10, ""clf__p"": 2}",0.548373,1.743708e+09,0.955173,0.951467,0.951467,0.895121,0.898917,0.880615,"{""tn"": 79, ""fn"": 17, ""tp"": 666, ""fp"": 84}"
8,BernoulliNB,"{""clf__alpha"": 1.0}",0.457339,1.743708e+09,0.956423,0.950095,0.950095,0.891701,0.947094,0.880615,"{""tn"": 69, ""fn"": 7, ""tp"": 676, ""fp"": 94}"
9,DummyClassifier,"{""clf__strategy"": ""most_frequent""}",0.445810,1.743708e+09,0.500000,0.500000,0.500000,0.577519,0.500000,0.807329,"{""tn"": 0, ""fn"": 0, ""tp"": 683, ""fp"": 163}"


### Tree

#### DecisionTreeClassifier

In [20]:
%%time
clf = DecisionTreeClassifier()

vectorizer = TfidfVectorizer()
pipe = Pipeline(steps = [('vectorizer',vectorizer),('clf',clf)])

parameter_grid = {
    "clf__splitter": ('best', 'random'),
    "clf__criterion": ('gini', 'entropy', 'log_loss'),
    "clf__max_features": ('sqrt', 'log2')
}
scoring = {"AUC": "roc_auc"}

gs = GridSearchCV(pipe,parameter_grid,scoring=scoring,refit='AUC', n_jobs=N_JOBS)
gs.fit(X_train, y_train)
save_best_params()

CPU times: total: 2.34 s
Wall time: 15.4 s


,name,best_params,mean_fit_time,timestamp,train_auc,train_accuracy,test_auc,test_accuracy,valid_auc,valid_accuracy,confusion
0,NuSVC,"{""clf__kernel"": ""rbf"", ""clf__nu"": 0.25, ""clf__...",19.793394,1.743712e+09,0.970584,0.968602,0.968602,0.919745,0.962193,0.904255,"{""tn"": 97, ""fn"": 15, ""tp"": 668, ""fp"": 66}"
1,SGDClassifier,"{""clf__alpha"": 1e-05, ""clf__loss"": ""log_loss"",...",0.549673,1.743709e+09,0.967128,0.967872,0.967872,0.911993,0.962193,0.905437,"{""tn"": 95, ""fn"": 12, ""tp"": 671, ""fp"": 68}"
2,LogisticRegression,"{""clf__C"": 10.0, ""clf__penalty"": ""l2""}",0.749055,1.743709e+09,0.967533,0.967745,0.967745,0.911537,0.962337,0.903073,"{""tn"": 93, ""fn"": 12, ""tp"": 671, ""fp"": 70}"
3,LinearSVC,"{""clf__C"": 0.1, ""clf__loss"": ""squared_hinge"", ...",0.464070,1.743709e+09,0.966799,0.965810,0.965810,0.908801,0.949636,0.881797,"{""tn"": 74, ""fn"": 11, ""tp"": 672, ""fp"": 89}"
4,RidgeClassifier,"{""clf__solver"": ""cholesky""}",11.230238,1.743709e+09,0.965955,0.965536,0.965536,0.911309,0.951208,0.893617,"{""tn"": 88, ""fn"": 15, ""tp"": 668, ""fp"": 75}"
5,ComplementNB,"{""clf__alpha"": 1.0}",0.543166,1.743708e+09,0.959284,0.955167,0.955167,0.893981,0.953058,0.880615,"{""tn"": 72, ""fn"": 10, ""tp"": 673, ""fp"": 91}"
6,MultinomialNB,"{""clf__alpha"": 1.0}",0.471005,1.743708e+09,0.959284,0.955167,0.955167,0.886001,0.953058,0.877069,"{""tn"": 61, ""fn"": 2, ""tp"": 681, ""fp"": 102}"
7,KNeighborsClassifier,"{""clf__n_neighbors"": 10, ""clf__p"": 2}",0.548373,1.743708e+09,0.955173,0.951467,0.951467,0.895121,0.898917,0.880615,"{""tn"": 79, ""fn"": 17, ""tp"": 666, ""fp"": 84}"
8,BernoulliNB,"{""clf__alpha"": 1.0}",0.457339,1.743708e+09,0.956423,0.950095,0.950095,0.891701,0.947094,0.880615,"{""tn"": 69, ""fn"": 7, ""tp"": 676, ""fp"": 94}"
9,DecisionTreeClassifier,"{""clf__criterion"": ""entropy"", ""clf__max_featur...",1.188098,1.743712e+09,0.833940,0.807126,0.807126,0.813953,0.667836,0.795508,"{""tn"": 75, ""fn"": 85, ""tp"": 598, ""fp"": 88}"


#### Extremely randomized tree 

In [21]:
%%time
clf = ExtraTreeClassifier()

vectorizer = TfidfVectorizer()
pipe = Pipeline(steps = [('vectorizer',vectorizer),('clf',clf)])

parameter_grid = {
    "clf__splitter": ('best', 'random'),
    "clf__criterion": ('gini', 'entropy', 'log_loss'),
    "clf__max_features": ('sqrt', 'log2')
}
scoring = {"AUC": "roc_auc"}

gs = GridSearchCV(pipe,parameter_grid,scoring=scoring,refit='AUC', n_jobs=N_JOBS)
gs.fit(X_train, y_train)
save_best_params()

CPU times: total: 2.59 s
Wall time: 16.5 s


,name,best_params,mean_fit_time,timestamp,train_auc,train_accuracy,test_auc,test_accuracy,valid_auc,valid_accuracy,confusion
0,NuSVC,"{""clf__kernel"": ""rbf"", ""clf__nu"": 0.25, ""clf__...",19.793394,1.743712e+09,0.970584,0.968602,0.968602,0.919745,0.962193,0.904255,"{""tn"": 97, ""fn"": 15, ""tp"": 668, ""fp"": 66}"
1,SGDClassifier,"{""clf__alpha"": 1e-05, ""clf__loss"": ""log_loss"",...",0.549673,1.743709e+09,0.967128,0.967872,0.967872,0.911993,0.962193,0.905437,"{""tn"": 95, ""fn"": 12, ""tp"": 671, ""fp"": 68}"
2,LogisticRegression,"{""clf__C"": 10.0, ""clf__penalty"": ""l2""}",0.749055,1.743709e+09,0.967533,0.967745,0.967745,0.911537,0.962337,0.903073,"{""tn"": 93, ""fn"": 12, ""tp"": 671, ""fp"": 70}"
3,LinearSVC,"{""clf__C"": 0.1, ""clf__loss"": ""squared_hinge"", ...",0.464070,1.743709e+09,0.966799,0.965810,0.965810,0.908801,0.949636,0.881797,"{""tn"": 74, ""fn"": 11, ""tp"": 672, ""fp"": 89}"
4,RidgeClassifier,"{""clf__solver"": ""cholesky""}",11.230238,1.743709e+09,0.965955,0.965536,0.965536,0.911309,0.951208,0.893617,"{""tn"": 88, ""fn"": 15, ""tp"": 668, ""fp"": 75}"
5,ComplementNB,"{""clf__alpha"": 1.0}",0.543166,1.743708e+09,0.959284,0.955167,0.955167,0.893981,0.953058,0.880615,"{""tn"": 72, ""fn"": 10, ""tp"": 673, ""fp"": 91}"
6,MultinomialNB,"{""clf__alpha"": 1.0}",0.471005,1.743708e+09,0.959284,0.955167,0.955167,0.886001,0.953058,0.877069,"{""tn"": 61, ""fn"": 2, ""tp"": 681, ""fp"": 102}"
7,KNeighborsClassifier,"{""clf__n_neighbors"": 10, ""clf__p"": 2}",0.548373,1.743708e+09,0.955173,0.951467,0.951467,0.895121,0.898917,0.880615,"{""tn"": 79, ""fn"": 17, ""tp"": 666, ""fp"": 84}"
8,BernoulliNB,"{""clf__alpha"": 1.0}",0.457339,1.743708e+09,0.956423,0.950095,0.950095,0.891701,0.947094,0.880615,"{""tn"": 69, ""fn"": 7, ""tp"": 676, ""fp"": 94}"
9,ExtraTreeClassifier,"{""clf__criterion"": ""entropy"", ""clf__max_featur...",1.247525,1.743712e+09,0.834454,0.832673,0.832673,0.837437,0.763557,0.855792,"{""tn"": 100, ""fn"": 59, ""tp"": 624, ""fp"": 63}"


### Ensemble

#### RandomForestClassifier

In [ ]:
%%time
clf = RandomForestClassifier()

vectorizer = TfidfVectorizer()
pipe = Pipeline(steps = [('vectorizer',vectorizer),('clf',clf)])

parameter_grid = {
    "clf__n_estimators": (10, 100, 500),
    "clf__criterion": ('gini', 'entropy', 'log_loss'),
    "clf__max_features": ('sqrt', 'log2')
}
scoring = {"AUC": "roc_auc"}

gs = GridSearchCV(pipe,parameter_grid,scoring=scoring,refit='AUC', n_jobs=N_JOBS)
gs.fit(X_train, y_train)
save_best_params()

CPU times: total: 4min 37s
Wall time: 31min 49s


#### AdaBoostClassifier

In [ ]:
%%time
clf = AdaBoostClassifier()

vectorizer = TfidfVectorizer()
pipe = Pipeline(steps = [('vectorizer',vectorizer),('clf',clf)])

parameter_grid = {
    "clf__n_estimators": (50, 100, 500),
    "clf__learning_rate": np.logspace(-5, 0, 6),
}
scoring = {"AUC": "roc_auc"}

gs = GridSearchCV(pipe,parameter_grid,scoring=scoring,refit='AUC', n_jobs=N_JOBS)
gs.fit(X_train, y_train)
save_best_params()

## Consolidar e exibir resultados

### Tempo de treinamento

In [ ]:
elapsed_time = datetime.now() - start_time
print('Total trainning time: {} seconds'.format(int(elapsed_time.total_seconds())))
print('Total trainning time: {} minutes'.format(int(elapsed_time.total_seconds()//60)))

### Scores

In [ ]:
df = load_best_params()
df

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,5))

# ax.set_xlim(0,1)
# ax.set_ylim(0,1)

for record in df.to_dict('records'):
    label = record['name']
    x = record['test_auc']
    y = record['test_accuracy']
    _=ax.scatter(x,y, label=label)

ax.set_xlabel('Área sob a curva ROC')
ax.set_ylabel('Acurácia')
ax.set_title('Desempenho dos classificadores')
ax.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,5))

# ax.set_xlim(0,1)
# ax.set_ylim(0,1)

for record in df.head(5).to_dict('records'):
    label = record['name']
    x = record['test_auc']
    y = record['test_accuracy']
    _=ax.scatter(x,y, label=label)

ax.set_xlabel('Área sob a curva ROC')
ax.set_ylabel('Acurácia')
ax.set_title('Desempenho dos 5 melhores classificadores')
ax.legend()
plt.show()